In [1]:
import pandas as pd
import time
from Functions import xbrl_dict,filings_collector
import datetime as dt
import requests
import xml.etree.ElementTree as ET
from pymongo import MongoClient

In [10]:
doc_date_str = "2019-03-16"
doc_date_dt = dt.datetime.fromisoformat(doc_date_str)
# doc_date_dt = dt.datetime.combine(doc_date_dt, dt.datetime.min.time())

# print(doc_date_dt)
type(doc_date_dt)

datetime.datetime